In [1]:
import torch
import math
from dataset import SatProblemDataSet
from models import MLP, LayerNormBasicLSTMCell, NeuroSAT
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [2]:
def compute_loss(logits, labels, parameters):
    # x = logits, z = labels
    # max(x, 0) - x * z + log(1 + exp(-abs(x)))
    predict_costs = torch.nn.functional.relu(logits) - logits * labels + torch.log1p(torch.exp(-torch.abs(logits)))
    predict_cost = torch.mean(predict_costs)
    l2_cost = torch.zeros([])
    for p in parameters:
        l2_cost += torch.sum(torch.square(p)) / 2
    return predict_cost + 1e-9 * l2_cost

In [3]:
n = NeuroSAT(5, 3, 3, 128)

In [4]:
ds = SatProblemDataSet("data")
dl = torch.utils.data.DataLoader(ds)

In [5]:
optim = torch.optim.Adam(n.parameters(), lr = 1e-4, weight_decay = 1e-10)
n.train()

NeuroSAT(
  (LC_msg): MLP(
    (linears): ModuleList(
      (0): Linear(in_features=5, out_features=5, bias=True)
      (1): Linear(in_features=5, out_features=5, bias=True)
      (2): Linear(in_features=5, out_features=5, bias=True)
      (3): Linear(in_features=5, out_features=5, bias=True)
    )
    (activation_func): ReLU()
  )
  (CL_msg): MLP(
    (linears): ModuleList(
      (0): Linear(in_features=5, out_features=5, bias=True)
      (1): Linear(in_features=5, out_features=5, bias=True)
      (2): Linear(in_features=5, out_features=5, bias=True)
      (3): Linear(in_features=5, out_features=5, bias=True)
    )
    (activation_func): ReLU()
  )
  (L_update): LayerNormBasicLSTMCell(
    (fiou_linear): Linear(in_features=15, out_features=20, bias=False)
    (fiou_ln_layers): ModuleList(
      (0): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
      (1): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
      (2): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
      (3):

In [9]:
for _ in range(1):
    for prob, target in dl:
        print(prob.type())
        print(type(target))
        optim.zero_grad()
        outputs = n(prob[0].float())
    #     print(outputs, prob[1])
        torch.nn.utils.clip_grad_norm_(n.parameters(), 0.5)
        loss = compute_loss(outputs, target[0], n.parameters())
        desc = 'loss: %.4f; ' % (loss.item())
        print(desc)

        loss.backward()
        optim.step()

torch.DoubleTensor
<class 'list'>


TypeError: only integer tensors of a single element can be converted to an index